In [2]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval

pd.options.mode.chained_assignment = None

In [3]:
data_path = "./data"
if os.listdir(data_path) != ['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']:
    print("[ERROR] Please download and unzip the dataset in a subdirectory './data'.")
else:
    print("[INFO] The dataset is correctly placed.")

[INFO] The dataset is correctly placed.


### Loading the data

In [4]:
tmdb_credits = pd.read_csv(os.path.join(data_path, "tmdb_5000_credits.csv"))
tmdb_credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [5]:
tmdb_movies = pd.read_csv(os.path.join(data_path, "tmdb_5000_movies.csv"))
tmdb_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [6]:
def parse_genres(genres):
    L=[]
    for genre in genres:
        L.append(genre['name'])
    if len(L) == 0:
        return np.nan
    return L


In [7]:
movie_genres = tmdb_movies[["genres", "release_date"]]
movie_genres.head()

,genres,release_date
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07


In [8]:
movie_genres["genres"] = movie_genres.genres.apply(lambda x: parse_genres(literal_eval(x)))
movie_genres.head()

,genres,release_date
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10
1,"[Adventure, Fantasy, Action]",2007-05-19
2,"[Action, Adventure, Crime]",2015-10-26
3,"[Action, Crime, Drama, Thriller]",2012-07-16
4,"[Action, Adventure, Science Fiction]",2012-03-07


In [9]:
movie_genres = movie_genres.dropna().explode("genres").reset_index().drop(columns="index")
movie_genres.head()

,genres,release_date
0,Action,2009-12-10
1,Adventure,2009-12-10
2,Fantasy,2009-12-10
3,Science Fiction,2009-12-10
4,Adventure,2007-05-19


In [10]:
movie_genres['release_date'] = pd.to_datetime(movie_genres['release_date'])
movie_genres['year'] = movie_genres['release_date'].apply(lambda x: x.strftime('%Y'))
movie_genres.head()

,genres,release_date,year
0,Action,2009-12-10,2009
1,Adventure,2009-12-10,2009
2,Fantasy,2009-12-10,2009
3,Science Fiction,2009-12-10,2009
4,Adventure,2007-05-19,2007


In [11]:
# Use only given years and drop TV Movies and Foreign genres

genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
movie_genres = movie_genres[movie_genres["year"].astype(int)>1965]
movie_genres = movie_genres[movie_genres["year"].astype(int)<2017]
movie_genres = movie_genres[~movie_genres["genres"].isin(genres_to_drop)]


In [12]:
df = movie_genres.groupby(["year","genres"],  as_index=False).count()
df.head()

,year,genres,release_date
0,1966,Action,1
1,1966,Adventure,1
2,1966,Comedy,2
3,1966,Crime,2
4,1966,Drama,2


In [13]:
df['norm_per_year'] = df['release_date'] / df.groupby('year')['release_date'].transform('sum')
df.head(40)

,year,genres,release_date,norm_per_year
0,1966,Action,1,0.066667
1,1966,Adventure,1,0.066667
2,1966,Comedy,2,0.133333
3,1966,Crime,2,0.133333
4,1966,Drama,2,0.133333
5,1966,Family,1,0.066667
6,1966,History,1,0.066667
7,1966,Music,1,0.066667
8,1966,Science Fiction,1,0.066667
9,1966,Thriller,2,0.133333


In [14]:
years = list(movie_genres.year.unique())
gens = list(movie_genres.genres.unique())

index = pd.MultiIndex.from_product([years, gens], names = ["year", "genres"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

In [16]:
zero_df.shape

(765, 4)

In [17]:
df = pd.concat([df,zero_df])

In [18]:
df.drop_duplicates(subset=['year', 'genres'], inplace=True, keep='first')

In [19]:
df.rename(columns={"release_date": "total_films"}, inplace=True)
df.head()

,year,genres,total_films,norm_per_year
0,1966,Action,1,0.066667
1,1966,Adventure,1,0.066667
2,1966,Comedy,2,0.133333
3,1966,Crime,2,0.133333
4,1966,Drama,2,0.133333


In [20]:
df.groupby("genres").count().shape

(15, 3)

In [21]:
df.sort_values(["year","genres"],inplace=True)

In [22]:
df.to_csv("mydata.csv")

#### Production houses

In [137]:
pc = tmdb_movies[["genres", "release_date", "production_companies"]]
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [138]:
def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L


In [139]:
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [140]:
pc["genres"] = pc.genres.apply(lambda x: parse_genres(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[Action, Adventure, Crime]",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [141]:
pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[Ingenious Film Partners, Twentieth Century Fo..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[Walt Disney Pictures, Jerry Bruckheimer Films..."
2,"[Action, Adventure, Crime]",2015-10-26,"[Columbia Pictures, Danjaq, B24]"
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[Legendary Pictures, Warner Bros., DC Entertai..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,[Walt Disney Pictures]


In [142]:
pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Ingenious Film Partners
1,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Twentieth Century Fox Film Corporation
2,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Dune Entertainment
3,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Lightstorm Entertainment
4,"[Adventure, Fantasy, Action]",2007-05-19,Walt Disney Pictures


In [143]:
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list

['Warner Bros.',
 'Universal Pictures',
 'Paramount Pictures',
 'Twentieth Century Fox Film Corporation',
 'Columbia Pictures',
 'New Line Cinema',
 'Metro-Goldwyn-Mayer (MGM)',
 'Touchstone Pictures',
 'Walt Disney Pictures',
 'Relativity Media']

In [144]:
pc = pc.dropna().explode("genres").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,Action,2009-12-10,Ingenious Film Partners
1,Adventure,2009-12-10,Ingenious Film Partners
2,Fantasy,2009-12-10,Ingenious Film Partners
3,Science Fiction,2009-12-10,Ingenious Film Partners
4,Action,2009-12-10,Twentieth Century Fox Film Corporation


In [145]:
pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

,genres,release_date,production_companies
0,Action,2009-12-10,Twentieth Century Fox Film Corporation
1,Adventure,2009-12-10,Twentieth Century Fox Film Corporation
2,Fantasy,2009-12-10,Twentieth Century Fox Film Corporation
3,Science Fiction,2009-12-10,Twentieth Century Fox Film Corporation
4,Adventure,2007-05-19,Walt Disney Pictures
...,...,...,...
5283,Drama,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5284,Romance,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5285,Action,1992-09-04,Columbia Pictures
5286,Crime,1992-09-04,Columbia Pictures


In [146]:
pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()

,genres,release_date,production_companies,year
0,Action,2009-12-10,Twentieth Century Fox Film Corporation,2009
1,Adventure,2009-12-10,Twentieth Century Fox Film Corporation,2009
2,Fantasy,2009-12-10,Twentieth Century Fox Film Corporation,2009
3,Science Fiction,2009-12-10,Twentieth Century Fox Film Corporation,2009
4,Adventure,2007-05-19,Walt Disney Pictures,2007


In [147]:

genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]
pc = pc[~pc["genres"].isin(genres_to_drop)]


In [148]:
df2 = pc.groupby(["year","production_companies","genres"],  as_index=False).count()
df2.head()

,year,production_companies,genres,release_date
0,1966,Twentieth Century Fox Film Corporation,Adventure,1
1,1966,Twentieth Century Fox Film Corporation,Comedy,1
2,1966,Twentieth Century Fox Film Corporation,Crime,1
3,1966,Twentieth Century Fox Film Corporation,Family,1
4,1966,Twentieth Century Fox Film Corporation,Science Fiction,1


In [149]:
df2['norm_per_year'] = df2['release_date'] / df2.groupby(['year','genres'])['release_date'].transform('sum')
df2.head()

,year,production_companies,genres,release_date,norm_per_year
0,1966,Twentieth Century Fox Film Corporation,Adventure,1,1.0
1,1966,Twentieth Century Fox Film Corporation,Comedy,1,1.0
2,1966,Twentieth Century Fox Film Corporation,Crime,1,0.5
3,1966,Twentieth Century Fox Film Corporation,Family,1,1.0
4,1966,Twentieth Century Fox Film Corporation,Science Fiction,1,1.0


In [150]:
years = list(pc.year.unique())
gens = list(pc.genres.unique())
comps = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years, gens, comps], names = ["year", "genres", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

In [151]:
zero_df

,year,genres,production_companies,release_date,norm_per_year
0,2009,Action,Twentieth Century Fox Film Corporation,0,0.0
1,2009,Action,Walt Disney Pictures,0,0.0
2,2009,Action,Columbia Pictures,0,0.0
3,2009,Action,Warner Bros.,0,0.0
4,2009,Action,Paramount Pictures,0,0.0
...,...,...,...,...,...
7645,1967,Music,New Line Cinema,0,0.0
7646,1967,Music,Metro-Goldwyn-Mayer (MGM),0,0.0
7647,1967,Music,Universal Pictures,0,0.0
7648,1967,Music,Relativity Media,0,0.0


In [152]:
# We must have all genres in all years, so we populate 0 where missing
df3 = pd.concat([df2,zero_df])
df3 = df3.drop_duplicates(subset=['year', 'genres','production_companies'], keep='first')

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [153]:
df3.shape

(7650, 5)

In [156]:
df3.rename(columns={"release_date": "total_films"}, inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
0,Adventure,1.0,Twentieth Century Fox Film Corporation,1,1966
1,Comedy,1.0,Twentieth Century Fox Film Corporation,1,1966
2,Crime,0.5,Twentieth Century Fox Film Corporation,1,1966
3,Family,1.0,Twentieth Century Fox Film Corporation,1,1966
4,Science Fiction,1.0,Twentieth Century Fox Film Corporation,1,1966


In [158]:
df3.sort_values(["year","genres", "production_companies"],inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
7352,Action,0.0,Columbia Pictures,0,1966
7356,Action,0.0,Metro-Goldwyn-Mayer (MGM),0,1966
7355,Action,0.0,New Line Cinema,0,1966
7354,Action,0.0,Paramount Pictures,0,1966
7358,Action,0.0,Relativity Media,0,1966


In [159]:
df3.to_csv("production_companies_stream.csv")